<a href="https://colab.research.google.com/github/chaw-thiri/DatabaseCourse/blob/main/DB_week_13_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# create a sample table
c.execute('''
CREATE TABLE IF NOT EXISTS employees(id INTEGER PRIMARY KEY, name TEXT, department TEXT, salary INTEGER)
''')

# Example 1: Efficient indexing
# ..............................................................
# Description: This example demonstrates how to create an index on the 'department' column.
# The index will improve the performance of queries filtering by department.

# adding sample data
sample_data = [(1, 'Alice', 'HR', 70000),
               (2, 'Bob', 'Engineer', 80000),
               (3, 'Charlie', 'HR', 75000)]
c.executemany('INSERT INTO employees VALUES(?, ?, ?, ?)', sample_data)  # Fixed the typo in 'executemany'

# create an index
c.execute('CREATE INDEX idx_department ON employees(department)')

# comment: Indexes can significantly improve the performance of queries that filter or sort based on the indexed column.
# however, they can also slow down data insertions and updates.
# Therefore, use them judiciously.

# querying with and without an index
c.execute('SELECT * FROM employees WHERE department = "HR"')
print("Query plan with index: ", c.fetchall())

# drop the index
c.execute('DROP INDEX IF EXISTS idx_department')  # Added 'IF EXISTS' to avoid an error if the index doesn't exist

c.execute('SELECT * FROM employees WHERE department="HR"')
print("Query plan without index: ", c.fetchall())

# clean up
conn.commit()
conn.close()


Query plan with index:  [(1, 'Alice', 'HR', 70000), (3, 'Charlie', 'HR', 75000)]
Query plan without index:  [(1, 'Alice', 'HR', 70000), (3, 'Charlie', 'HR', 75000)]


In [8]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# create a sample table
c.execute('''
CREATE TABLE IF NOT EXISTS products(id INTEGER PRIMARY KEY, name TEXT, category TEXT, price INTEGER)
''')

# adding sample data
sample_data = [(1, 'Laptop', 'Electronics', 1200),
               (2, 'Chair', 'Furniture', 150),
               (3, 'Smartphone', 'Electronics', 800)]
c.executemany('INSERT INTO products VALUES(?, ?, ?, ?)', sample_data)

# Query optimization example
# -----------------------------------------------------------
# Description: This example demonstrates how to write an optimized query by selecting only the necessary columns and using efficient filtering

# Efficient query: selecting specific columns and filtering
c.execute('SELECT name, price FROM products WHERE category = "Electronics"')
print("Efficient query results:", c.fetchall())

# Cleanup
conn.commit()
conn.close()


Efficient query results: [('Laptop', 1200), ('Smartphone', 800)]


In [9]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# create a sample table
c.execute('''
CREATE TABLE IF NOT EXISTS orders(id INTEGER PRIMARY KEY, product_id INTEGER, quantity INTEGER)
''')

# transaction optimization example
# -------------------------------------------------
# description : this example demonstrates the use of transactions to optimize bulk data insertion

# start a transaction
conn.execute(
    'BEGIN TRANSACTION '
)

# burt insert data
bulk_data = [(i,i%5+1, i * 2) for i in range(1,1001)]
c.executemany('INSERT INTO orders VALUES(?,?,?)',bulk_data)

conn.commit()

c.execute('SELECT COUNT(*) FROM orders')
print('Numberr of records inserted: ',c.fetchone()[0])

conn.close()


Numberr of records inserted:  1000


In [12]:
import sqlite3


# connect to sqlite database
conn = sqlite3.connect('example.db')
c = conn.cursor()


# create sample tables
c.execute("""CREATE TABLE IF NOT EXISTS students (id INTEGER PRIMARY KEY, name TEXT, class_id INTEGER)""")
c.execute('''CREATE TABLE IF NOT EXISTS classes (id INTEGER PRIMARY KEY,class_name TEXT)''')


# adding some sample data
students_data= [(1,'John',1),(2,'Emma',2),(3,'Harry',1)]
classes_data = [(1,'Math'),(2,'Science')]
c.executemany('INSERT INTO students VALUES(?,?,?)',students_data)
c.executemany('INSERT INTO classes VALUES(?,?)',classes_data)
# efficient joins example
# -------------------------------------------
# description : this example demonstratese how to perform an efficient join between two tables

# effieient join query
c.execute('''SELECT students.name, classes.class_name
            FROM students
            JOIN classes ON students.class_id = classes.id
''')

print("Join results: ",c.fetchall())


# clean up
conn.commit()
conn.close()


Join results:  [('John', 'Math'), ('Emma', 'Science'), ('Harry', 'Math')]


In [14]:
import sqlite3

# connect to sqlite datbase
conn = sqlite3.connect('example.db')
c = conn.cursor()


# advanced PRAGMA optimization example
# ----------------------------------------------------
# description : this example demonstares the advanced use of PRAGMA statements
# for various database optimization including write-ahead logging, foregin key enforcement,
# and query performance improvement

# 1. setting the journal mode to WAL(write-ahead logging)
# wal can improve the performance of read opertions while wrtie operations are happening
c.execute('PRAGMA journal_mode = WAL')
journal_mode = c.fetchone()[0]
print("Journal mode set to: ", journal_mode)

# 2: enabling foreign key constraint
# ensures referential integrity in the database
c.execute('PRAGMA foreign_keys = ON')
c.execute('PRAGMA foreign_keys')
fk_status = c.fetchone()[0]
print("Foreign key status :  ", "Enabled" if fk_status else "Disabled")


# 3. setting synchronous mode to normal
# this reduces the number of disk writes, speeding up transactions at the risk of some data loss in case of a crash,
c.execute('PRAGMA synchronous=NORMAL')
c.execute('PRAGMA synchronous')
sync_mode = c.fetchone()[0]
print("Synchronous mode set to ", sync_mode)

#4. adjusting cache size
# increasing the cache size can improve the database read performance by reducing disk I/O
c.execute('PRAGMA cache_size= 10000') # setting cache size to 10000pages
c.execute("PRAGMA cache_size")
cache_size = c.fetchone()[0]
print("Cache size set to:", cache_size,"pages")

# 5. querying the database file
# useful for monitoring the database size, which can be a factor in performanc.
c.execute('PRAGMA page_size')
page_size = c.fetchone()[0]
c.execute('PRAGMA page_count')
page_count = c.fetchone()[0]
db_size = page_size * page_count
print("Database size: ", db_size, "bytes")

# clean up
conn.close()

Journal mode set to:  wal
Foreign key status :   Enabled
Synchronous mode set to  1
Cache size set to: 10000 pages
Database size:  36864 bytes
